In [2]:
np.object = object
np.int = int
np.float = float
np.bool = bool

In [1]:
pip install numpy

  Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl (14.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konlpy 0.5.2 requires beautifulsoup4==4.6.0, but you have beautifulsoup4 4.11.1 which is incompatible.


In [2]:
pip install tensorflow

     ---------------------------------------- 0.0/272.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/272.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/272.8 MB 3.8 MB/s eta 0:01:12
     ---------------------------------------- 0.2/272.8 MB 2.9 MB/s eta 0:01:34
     ---------------------------------------- 0.5/272.8 MB 3.6 MB/s eta 0:01:16
     ---------------------------------------- 0.6/272.8 MB 3.6 MB/s eta 0:01:16
     ---------------------------------------- 0.6/272.8 MB 3.6 MB/s eta 0:01:16
     ---------------------------------------- 0.6/272.8 MB 3.6 MB/s eta 0:01:16
     ---------------------------------------- 0.8/272.8 MB 2.8 MB/s eta 0:01:37
     ---------------------------------------- 0.8/272.8 MB 2.8 MB/s eta 0:01:37
     ---------------------------------------- 1.2/272.8 MB 3.0 MB/s eta 0:01:31
     ---------------------------------------- 1.2/272.8 MB 2.9 MB/s eta 0:01:34
     ---------------------------------------- 1.2/272.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konlpy 0.5.2 requires beautifulsoup4==4.6.0, but you have beautifulsoup4 4.11.1 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import FinanceDataReader as fdr
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping

ImportError: this version of pandas is incompatible with numpy < 1.20.3
your numpy version is 1.19.5.
Please upgrade numpy to >= 1.20.3 to use this pandas version

In [3]:
KP_stock_list = fdr.StockListing("KOSPI")
df=KP_stock_list
df

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,65500,1,900,1.39,65200,65900,65000,19699481,1288159418828,391020757025000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,581000,2,-6000,-1.02,590000,591000,573000,316880,183868381000,135954000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,89500,1,700,0.79,90800,91400,88700,3737493,336529892700,65156211667500,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,781000,1,3000,0.39,775000,787000,775000,51554,40277366000,55586894000000,71174000,STK
4,051910,KR7051910008,LG화학,KOSPI,,740000,2,-1000,-0.13,746000,750000,730000,257164,189764736000,52238333820000,70592343,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,000545,KR7000541003,흥국화재우,KOSPI,,5610,1,30,0.54,5500,5630,5500,3775,20886220,4308480000,768000,STK
944,002787,KR7002782001,진흥기업2우B,KOSPI,,13890,1,370,2.74,13520,13890,13400,63,858540,4094883120,294808,STK
945,001527,KR7001522002,동양2우B,KOSPI,,13200,1,50,0.38,13190,13220,13000,374,4877960,4073110800,308569,STK
946,001525,KR7001521004,동양우,KOSPI,,6540,1,70,1.08,6500,6540,6470,637,4149960,4039960740,617731,STK


In [4]:
# 코스피100 종목 리스트
stock_list = ['삼성전자', 'SK하이닉스', '현대차', '셀트리온', 'POSCO', '삼성바이오로직스', 'LG화학', 'NAVER', '삼성SDI', '삼성물산']

# 각 종목의 데이터를 불러옵니다.
data = {}
for stock in stock_list:
    dff = df[df.Name==stock]
    data[stock] = dff

In [5]:
dff

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
15,028260,KR7028260008,삼성물산,KOSPI,,109600,1,2500,2.33,107600,109600,107600,367406,40014478000,20482824077600,186887081,STK


In [6]:
# 예측 기간 설정
pred_range = 30

# 학습에 사용할 데이터셋 생성
def create_dataset(df, pred_range):
    x_data, y_data = [], []
    for i in range(len(df) - pred_range):
        x = df.iloc[i:(i + pred_range), :]
        y = df.iloc[(i + pred_range), 3]
        x_data.append(x.values)
        y_data.append(y)
    return np.array(x_data), np.array(y_data)

x_train = {}
y_train = {}

scaler= MinMaxScaler()

# 스케일을 적용할 column을 정의합니다.
scale_cols = [ 'Close','Open', 'High', 'Low', 'Volume']

for stock in stock_list:
    df_scaled = scaler.fit_transform(data["삼성물산"][scale_cols])
    x_train[stock], y_train[stock] = create_dataset(df_scaled, pred_range)


In [7]:
# LSTM 모델 생성
def create_model():
    model = Sequential()
    model.add(LSTM(64, input_shape=(pred_range, 5), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    return model

# 각 종목별 모델 생성 및 학습
models = {}
for stock in stock_list:
    model = create_model()
    early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    model.fit(x_train["삼성물산"], y_train[stock], epochs=50, batch_size=16, validation_split=0.1, callbacks=[early_stop])
    models["삼성물산"] = model

# 각 종목별 예측
pred


NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
# 각 종목별 예측
def predict(model, df_scaled):
    x_test = []
    for i in range(len(df_scaled) - pred_range):
        x = df_scaled[i:(i + pred_range), :]
        x_test.append(x)
    x_test = np.array(x_test)
    return model.predict(x_test)

pred = {}
for stock in stock_list:
    df_scaled = scaler.fit_transform(data[stock][scale_cols])
    y_pred = predict(models[stock], df_scaled)
    pred[stock] = y_pred


In [ ]:
import matplotlib.pyplot as plt

# 예측 결과 시각화
def plot_prediction(stock, data, pred):
    plt.figure(figsize=(15, 6))
    plt.plot(data['Date'][pred_range:], data['Close'][pred_range:], label='actual')
    plt.plot(data['Date'][pred_range:], pred, label='prediction')
    plt.title(f'{stock} Stock Price Prediction')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

for stock in stock_list:
    df = data[stock][pred_range+1:]
    y_pred = scaler.inverse_transform(pred[stock])
    plot_prediction(stock, df, y_pred)
